# Modelo Condensación - Wikramanayake et al.
   

Implementa algunas fórmulas para describir la transferencia de calor en la condensación por gotas, teniendo en cuenta la distribución de tamaños de gotas, basado en los trabajos de Wrikamanayake (2019) y KimKim (2012) 

Se trata de un conjunto de resistencias térmicas que se deben vencer para la transferencia de calor.

In [52]:
## Importar librerías 


import math
import numpy as np
import scipy
import scipy.integrate as integrate
from functools import partial


### Nomenclatura&nbsp;




**R_dif**=resistencia térmica por la capa de difusión

**T_amb**=Temperatura ambiente [K]

**T_int**=Temperatura de la gota [K]

**t_c**=Tasa de condensación por área. Hallada experimentalmente [Kg/m²s]

**h_fg**= Calor latente de condensación [J/kg]

**K_aire**=Conductividad térmica del K_aire [W/m K]

**delta**= grosor de la capa límite [m]

**S_r**=Área proyectada de la gota [m²]

**R_g**: Constante específica de los gases J/[Kg K]

**v_g**: Volumen especifico del vapor de agua [m²/Kg]

**h_fg**: Calor latente de vaporización [J/Kg]

**alfa**=Coeficiente de acomodación o condensación; fracción de moléculas de vapor que se mueven en la gota líquida durante el cambio de fase. Debe ser de entre 0.02 a 0.04 para agua. En Wrikamanayake et al. se usó 0.02

**T_s**=Temperatura de superficie de pared [K]

**k1**=Conductividad térmica del agua


**r**=radio de la gota [m]

**theta**=Ángulo de contacto [°]


Se hacen las resistencias desde la parte exterior hasta el sustrato

### Parámetros de entrada

In [50]:
T_amb=294.1
T_int=273
t_c=2.7e-06 # Kg/m²s tomado del estado estable para 0V en wrikamaniyake et al
h_fg=2264705
k_aire=1
delta=0.00001
R_g=286.9 #Solo aire
v_g=1
alfa=0.02
T_s=273
theta=90

r=0.0015
k1=0.58

r_min: 3.8961625003973076e-09 # [m]
r_max: 0.0010561535753410094 # [m]


In [51]:
def R_dif_Wrikamanayake (T_amb, T_int, t_c, h_fg):
#----------------Resistencia a la difusión del vapor en la capa de no condensables [K/W]----------------
    R_dif=(T_amb-T_int)/(t_c*h_fg)
    return R_dif

def R_cond_aire_Wrikamanayake (r,K_aire, delta):
#--------------Resistencia a la conducción del aire [K/W]---------------------
    R_cond_aire=delta/(k_aire*math.pi*(r**2))
    return R_cond_aire


def R_int_liq_Wrikamanayake (r, theta, alfa, T_s):

#--Definición de h_i (coeficiente interfacial)
#h_i=Coeficiente interfacial de transferencia de Calor

    h_i=((2*alfa)/(2-alfa))*(1/math.sqrt(2*math.pi*R_g*T_s))*((h_fg**2)/v_g*T_s)

#--------Resistencia de Interfase líquido-gas [K/W]----------------------------

    R_int=1/(h_i*2*math.pi*(r**2)*(1-math.cos(theta)))

    return R_int

def R_cond_gotas_Wrikamanayake(r,theta, k1):
#-----Resistencia a la conducción de las gotas [K/W]-------

    R_drop=theta/(4*math.pi*k1*r*math.sin(theta))

    return R_drop

#-----------RESISTENCIA TÉRMICA TOTAL [K/W]---------------

R_dif=R_dif_Wrikamanayake(T_amb, T_int ,t_c, h_fg)
R_cond_aire=R_cond_aire_Wrikamanayake(r,K_aire, delta)
R_int= R_int_liq_Wrikamanayake (r, theta, alfa, T_s)
R_drop=R_cond_gotas_Wrikamanayake(r, theta, k1)

R_total=(((1/R_dif)+(1/R_cond_aire))**(-1))+R_int+R_drop

#-----TRANSFERENCIA DE CALOR DEPENDIENTE DEL RADIO DE LA GOTA----------

q=(T_amb-T_s)/R_total


# Definir funciones para la tasa de transferencia de calor

    R_cond_dif = partial(R_cond_aire_Wrikamanayake, K_aire=K_aire, delta=delta)
    R_cond_dif.__doc__ = "Resistencia para cada gota dependiente del radio r en m"  
   
    !!!!!!

    n = partial(n_KimKim, deltaT_sub=deltaT_sub, r_min=r_min, delta_coat=delta_coat, k_coat=k_coat, k_c=k_c,
                Theta=Theta, h_i=h_i, rho=rho, h_fg=h_fg, r_e=r_e, r_max=r_max)
    n.__doc__ = "drop size distribution for small drops depending on drop radius r in m"
    N = partial(N_LeFevre, r_max=r_max)
    N.__doc__ = "drop size distribution for large drops depending on drop radius r in m"

    # integrate and calculate heat flux density
    def Q_drop_n(r):
        '''small drops'''
        Q_drop_n = Q_drop(r) * n(r)
        return Q_drop_n

    def Q_drop_N(r):
        '''large drops'''
        Q_drop_N = Q_drop(r) * N(r)
        return Q_drop_N


    def N_LeFevre(r, r_max):
    """ Distribución de tamaño de gotas para gotas grandes """
    N = 1/(3*math.pi*r**2*r_max) * (r/r_max)**(-2/3)
    return N     
    
    def n_KimKim(r, deltaT_sub, r_min, delta_coat, k_coat, k_c, Theta, h_i, rho, h_fg, r_e, r_max):
    """ Distribución de tamaño de gotas para gotas pequeñas """
    A_1 = deltaT_sub / (2*rho*h_fg)
    A_2 = Theta * (1-math.cos(Theta)) / (4*k_c*math.sin(Theta))
    A_3 = 1/(2*h_i) + delta_coat*(1-math.cos(Theta)) / (k_coat*(math.sin(Theta))**2)
    tau = 3*r_e**2 * (A_2*r_e + A_3)**2 / (A_1*(11*A_2*r_e**2 - 14*A_2*r_e*r_min + 8*A_3*r_e - 11*A_3*r_min))
    B_1 = A_2/(tau*A_1) * ((r_e**2-r**2)/2 + r_min*(r_e-r) - r_min**2*math.log((r-r_min)/(r_e-r_min))) 
    B_2 = A_3/(tau*A_1) * (r_e-r - r_min*math.log((r-r_min)/(r_e-r_min)))
    n = 1/(3*math.pi*r_e**3*r_max) * (r_e/r_max)**(-2/3) * r*(r_e-r_min)/(r-r_min) * \
        (A_2*r+A_3)/(A_2*r_e+A_3) * math.exp(B_1+B_2)
    return n 
    
    
print ("R_dif:", R_dif,"\n"
       "R_cond_aire:",R_cond_aire,"\n"
      "R_int:", R_int,"\n"
       "R_drop:",R_drop,"\n"
      "R_total",R_total,"\n")

print("Calor dependiente del radio:", q,"W\n")



 
##-----FLUJO DE CALOR EN LA SUPERFICIE CONOCIENDO LA DISTRIBUCIÓN DE GOTAS
#
# from scipy.integrate import quad
#
# def f(r):
#     return q(r)*N(r) # está incompleta
#
# i, err= quad(f, r_min, r_max)
# print (i)
# print (err)


R_dif: 3.4506987951255566 
R_cond_aire: 2.2488593458884812e-06 
R_int: 1.2114424248146883e-06 
R_drop: 0.6238566845690902 
R_total 0.6238601448693952 

Calor dependiente del radio: 33.8216829100652 W

